## Description of what we are going to do

create_mangrove_wdpa_table.js file

In [1]:
import os
from pathlib import Path
import geopandas
import zipfile
import subprocess
import logging
import requests
from typing import Union
from ipyleaflet import Map, GeoData, basemaps, LayersControl

In [2]:
!npm install -g mapshaper@latest

              ] \ reify:flatbush: http fetch GET 200 https://registry.npmj.n
added 65 packages, and audited 66 packages in 8s

6 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities


In [30]:
#  FIXME: This will depends from where the notebook kernel is running so be carefull
WORK_DIR =Path(os.getcwd())
BASE_DIR = f'{WORK_DIR}/work/datasets/raw'
logging.basicConfig(level=logging.INFO)

# @TODO: Add expected data files source as an environment variable.
assert BASE_DIR == '/home/jovyan/work/datasets/raw', f'{BASE_DIR} is not the correct directory'

In [31]:
def download_wdpaData(file_path: str, update: bool = False) -> Union[int, str]:
    """
    Download a WDPA file to a path.
    Parameters
    ----------
    file_path : str - The path to the file to download.
    update : bool, optional - If True, the file will be downloaded again even if it already exists.
                            The default is False.
    
    Returns
    -------
    int - 0 if the file was downloaded successfully, 1 if the file download failed.
    """
    try:
        if update or not os.path.exists(file_path):
            logging.info('Downloading WDPA data...')
            
            url_info = requests.post('https://www.protectedplanet.net/downloads',
                                data={"domain":"general",
                                    "format":"shp",
                                    "token":"wdpa",
                                    "id":51216}
                                )
            
            response = requests.get(url_info.json()['url'], stream=True)
        
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=128):
                    f.write(chunk)
        else:
            logging.info('WDPA data already downloaded.')    
        
        return 0
    except Exception as e:
        logging.error(e)
        return 1

In [32]:
def extract_wdpa_data(wdpa_zip_file: str, target_folder: str, format: str='.zip', update: bool = False) -> int:

    """
    Extract WDPA data from a zip file.
    Parameters
    ----------
    wdpa_zip_file : str - The path to the zip file containing the WDPA data.
    target_folder : str - The path to the folder to extract the WDPA data to.
    format : str, optional - The format of the zip file. The default is '.zip'.
    update : bool, optional - If True, the file will be extracted even if it already exists.
                            The default is False.
    
    Returns
    -------
    int - 0 if the file was extracted successfully, 1 if the file extraction failed.

    """
    try:
        path = f'{BASE_DIR}/{target_folder}'

        if update or not os.path.exists(path):
            logging.info('Extracting WDPA data...')
               
            if not os.path.exists(path):
                os.mkdir(path)

            with zipfile.ZipFile(wdpa_zip_file, 'r') as zip_ref:
                sublist = filter(lambda file: format in file, zip_ref.namelist())
                zip_ref.extractall(path, members=sublist)
        
            for file in os.listdir(path):
                if file.endswith(format):
                    logging.info(file)
                    
                    inner_folder = f'{path}/{file[:-len(format)]}'
                    
                    with zipfile.ZipFile(f'{path}/{file}', 'r') as zip_ref:
                        if not os.path.exists(inner_folder):
                            os.mkdir(inner_folder)

                        zip_ref.extractall(inner_folder)
                    
                    os.remove(f'{path}/{file}')
        else:
            logging.info('WDPA data already extracted.')
            
        return 0
    
    except Exception as e:
        logging.error(e)
        return 1
        

In [41]:
def load_wdpa_data(file_path: str) -> Union[int, geopandas.GeoDataFrame]:
    """
    Load WDPA data from a csv file.

    Parameters
    ----------
    file_path : str - The path to the csv file to load.

    Returns
    -------
    geopandas.GeoDataFrame - The loaded data.

    """
    try:
        return geopandas.read_file(file_path)
    except Exception as e:
        logging.error(e)
        return 1

In [46]:
def simplifyMapshaper(data_path: Path, output_path: Path, 
    simplification: bool, update: bool = False) -> Union[int, geopandas.GeoDataFrame]:
    """
    Simplify geometry of a GeoDataFrame using mapshaper.
    Parameters
    ----------
    data_path : Path - The path to the GeoDataFrame to simplify.
    output_path : Path - The path to the output GeoDataFrame.
    simplification : bool - If True, the data will be simplified.
    update : bool, optional - If True, the output GeoDataFrame will be overwritten.
                            The default is False.
    
    Returns
    -------
    int - 0 if the data was simplified successfully, 1 if the data simplification failed.
    """
    try:
        if update or not os.path.exists(output_path):
            
            logging.info('Filtering WDPA data based on Ramsar designation and in active protection...')
            
            subprocess.run(f"mapshaper-xl 16gb -i {data_path} combine-files snap \
                -filter \'STATUS!=\"Proposed\"\' \
                -filter bbox=-180,-50,180,50 remove-empty \
                -clip source={BASE_DIR}/mangrove_coverage-v2.shp target=* \
                -merge-layers target=*\
                -clean rewind allow-overlaps \
                -o {output_path} format=shapefile force", shell=True, check=True)
        
        if simplification:
            logging.info('Simplifying WDPA data...')

            subprocess.run(f"mapshaper-xl 16gb -i {output_path} \
                -simplify 50% visvalingam keep-shapes planar \
                -filter-islands min-vertices=3 min-area=10000m2 remove-empty \
                -filter-slivers min-area=10000m2 remove-empty \
                -clean rewind \
                -o {output_path} format=shapefile force", shell=True, check=True)
        
        return load_wdpa_data(output_path)
    
    except Exception as e:
        logging.error(e)
        return 1

In [34]:
def mapRender(data: geopandas.GeoDataFrame) -> Map:
    """
    Render a map with the given data.
    Parameters
    ----------
    data : geopandas.GeoDataFrame - The data to render.

    Returns
    -------
    Map - The rendered map.
    """
    try:
        m = Map(center=(52.3,8.0), 
            zoom = 3, 
            basemap= basemaps.Esri.WorldTopoMap,
            layers_control=True,
            controls=[
                LayersControl()
            ])
        geo_data = GeoData(
                    geo_dataframe=data,
                    style={
                        'color': '#ff0000',
                        'fillOpacity': 0.5
                    },
                    name='WDPA',
                )
        m.add_layer(geo_data)

        
        return m
    except Exception as e:
        logging.error(e)
        return 1

In [35]:
download_wdpaData(f'{BASE_DIR}/wdpa_protected_areas_public.zip')

INFO:root:WDPA data already downloaded.


0

In [36]:
extract_wdpa_data(f'{BASE_DIR}/wdpa_protected_areas_public.zip', 'wdpa')

INFO:root:WDPA data already extracted.


0

In [37]:
outFolder= Path(f'{BASE_DIR}/wdpa')
paths =[]
for root, dirs, files in os.walk(outFolder):
    for file in files:
        if file.endswith("-polygons.shp"):
            paths.append(os.path.join(root, file))

path = ' '.join(paths)

outputPath = f'{outFolder.parents[1]}/processed/wdpa_protected_areas_public.shp'
print(outputPath)

/home/jovyan/work/datasets/processed/wdpa_protected_areas_public.shp


In [47]:
wdpa = simplifyMapshaper(path, outputPath, simplification=False, update=True)

INFO:root:Filtering WDPA data based on Ramsar designation and in active protection...
Allocating 16 GB of heap memory
[i] Snapped 19910585 points
[filter] Retained 85,479 of 85,785 features
[filter] Retained 85,513 of 85,784 features
[filter] Retained 85,140 of 85,785 features
[filter] Retained 48,097 of 85,479 features
[filter] Retained 57,392 of 85,513 features
[filter] Retained 43,671 of 85,140 features
[clean] Retained 20,767 of 20,767 features
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_protected_areas_public.shp
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_protected_areas_public.shx
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_protected_areas_public.dbf
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_protected_areas_public.prj


In [44]:
## @usage: based on by single processing of the files.
# for idx, inputPath in enumerate(paths):
#     simplifyMapshaper(inputPath, 
#     f'{outFolder.parents[1]}/processed/wdpa_protected_areas_public_{idx}.shp', 
#     simplification=False, 
#     update=True)

INFO:root:Filtering WDPA data based on Ramsar designation and in active protection...
Allocating 16 GB of heap memory
[i] Snapped 3820009 points
[filter] Retained 85,479 of 85,785 features
[clean] Retained 3,077 of 3,077 features
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_protected_areas_public_0.shp
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_protected_areas_public_0.shx
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_protected_areas_public_0.dbf
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_protected_areas_public_0.prj
INFO:root:Filtering WDPA data based on Ramsar designation and in active protection...
Allocating 16 GB of heap memory
[i] Snapped 3620576 points
[filter] Retained 85,513 of 85,784 features
[clean] Retained 7,172 of 7,172 features
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_protected_areas_public_1.shp
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_protected_areas_public_1.shx
[o] Wrote /home/jovyan/work/datasets/processed/wdpa_pr

In [32]:
wdpa.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2115 entries, 0 to 2114
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   WDPAID      2115 non-null   int64   
 1   WDPA_PID    2115 non-null   object  
 2   PA_DEF      2115 non-null   object  
 3   NAME        2115 non-null   object  
 4   ORIG_NAME   2115 non-null   object  
 5   DESIG       2115 non-null   object  
 6   DESIG_ENG   2115 non-null   object  
 7   DESIG_TYPE  2115 non-null   object  
 8   IUCN_CAT    2115 non-null   object  
 9   INT_CRIT    2115 non-null   object  
 10  MARINE      2115 non-null   object  
 11  REP_M_AREA  2115 non-null   float64 
 12  GIS_M_AREA  2115 non-null   float64 
 13  REP_AREA    2115 non-null   float64 
 14  GIS_AREA    2115 non-null   float64 
 15  NO_TAKE     2115 non-null   object  
 16  NO_TK_AREA  2115 non-null   float64 
 17  STATUS      2115 non-null   object  
 18  STATUS_YR   2115 non-null   int64   
 19

In [ ]:
mapRender(wdpa)

In [ ]:
# TODO: add a function to zip data and upload it to google cloud storage bucket